In [ ]:
!pip3 install git+https://github.com/PipableAI/pip-flow.git

In [2]:
from pip_flow import PipFlow

## Intitialise the class and model

By default, when initializing the `PipEtl` class without passing any device parameter, the model inference will require a GPU with at least 10-12 GB of VRAM.

```python
generator = PipEtl(device = 'cuda')
```

To test out the model without the need for a GPU, you can utilize the model hosted at PipableAI's end by initializing the model with "cloud" as the value for the device parameter.






In [3]:
pip_flow = PipFlow(device = 'cloud', model_key = "PipableAI/pip-code-bandit")

## Generate from a prompt [Miscellaneous Examples]

In [4]:
prompt = """
<story>
There once was a boy who grew bored while watching over the village sheep. He wanted to make things more exciting. So, he yelled out that he saw a wolf chasing the sheep. All the villagers came running to drive the wolf away. However, they saw no wolf. The boy was amused, but the villagers were not. They told him not to do it again. Shortly after, he repeated this antic. The villagers came running again, only to find that he was lying. Later that day, the boy really sees a wolf sneaking amongst the flock. He jumped up and called out for help. But no one came this time because they thought he was still joking around. At sunset, the villagers looked for the boy. He had not returned with their sheep. They found him crying. He told them that there really was a wolf, and the entire flock was gone. An old man came to comfort him and told him that nobody would believe a liar even when they are being honest.
</story>
<question>
Give a short summary of the above story.
</question>
"""

print(pip_flow.generate(prompt, eos_token="response", max_new_tokens=100))


The story begins with a boy who grows bored and watches over a village sheep. He yells to a wolf to chase the sheep, but no wolf is seen. The boy tells the villagers to not do it again. Later, the boy sees a wolf sneaking amongst the flock, and he calls for help. The villagers tell him that the boy is lying, but he is still lying. The story ends with the boy telling the villagers that there is a


In [5]:
prompt = """
<data>
a = 1, 3 and 5
b = 'John' and 'Michael'
</data>
<question>
Create a XML file with the data above.
</question>
"""

print(pip_flow.generate(prompt, eos_token="xml", max_new_tokens=50))


<data>
<a>1, 3 and 5</a>
<b>John and Michael</b>
</data>



In [6]:
prompt = """
<data>
a = 1, 3 and 5
b = 'John' and 'Michael'
</data>
<question>
Create a JSON file with the data above.
</question>
"""

print(pip_flow.generate(prompt, eos_token="json", max_new_tokens=50))


{
  "a": [1, 3, 5],
  "b": ['John', 'Michael']
}



In [7]:
prompt = """
<question>
write a python function to double the input number.
</question>
"""

print(pip_flow.generate(prompt, eos_token="code", max_new_tokens=50))




```python
def double_number(n):
  return n * 2
```




In [8]:
prompt = """
<json>
{
  "a": [1, 3, 5],
  "b": ['John', 'Michael']
}
</json>
<question>
Parse the json to xml.
</question>
"""

print(pip_flow.generate(prompt, eos_token="xml", max_new_tokens=50))


<a>[1, 3, 5]</a>
<b>John, Michael</b>



## Generate SQL

In this example, we will attempt to generate a SQL query. We will explore various examples demonstrating how to utilize instructions and examples for a more precise and accurate response.

In [30]:
schema = f"""
CREATE TABLE department (Department_ID number,
  Name text,
  Creation text,
  Ranking number,
  Budget_in_Billions number,
  Num_Employees number);

CREATE TABLE head (head_ID number,
  name text,
  born_state text,
  age number);

CREATE TABLE management (department_ID number,
  head_ID number,
  temporary_acting text);
"""

question = "What are the names of the heads who are born outside the California state ?"

query = pip_flow.generate_sql(
    schema=schema, question=question,
)

SELECT name FROM head WHERE born_state != 'California';

### Use of instructions parameter

By utilizing the `instruction` parameter, you have the flexibility to provide supplementary context directly relevant to your question.

In [31]:
schema = f"""
CREATE TABLE department (Department_ID number,
  Name text,
  Creation text,
  Ranking number,
  Budget_in_Billions number,
  Num_Employees number);

CREATE TABLE head (head_ID number,
  name text,
  born_state text,
  age number);

CREATE TABLE management (department_ID number,
  head_ID number,
  temporary_acting text);
"""

instructions = """
1. In department table, column Budget_in_Billions is in billions, so numeric 1 will represent 1 billion.
"""

question = "What are the names of department with budget more than 3 billions ?"


query = pip_flow.generate_sql(schema=schema, question=question, instructions=instructions)


SELECT Name FROM department WHERE Budget_in_Billions > 3;

### Use of Example Parameter

For complex queries or queries requiring specific jargons and query formats, you can provide them as examples. Then, you can ask the model similar questions, which will result in the generation of appropriate and accurate queries.

In [32]:
schema = f"""
CREATE TABLE department (Department_ID number,
  Name text,
  Creation text,
  Ranking number,
  Budget_in_Billions number,
  Num_Employees number);

CREATE TABLE head (head_ID number,
  name text,
  born_state text,
  age number);

CREATE TABLE management (department_ID number,
  head_ID number,
  temporary_acting text);
"""

instructions = """
1. In department table, column Budget_in_Billions is in billions, so numeric 1 will represent 1 billion.
"""

examples = """
--question: Fetch the name of the departments, the ages of their heads, and whether there is any temporary acting head for departments with a budget exceeding $1 billion.
--sql:
SELECT d.Name AS Department_Name, h.age AS Head_Age,
    CASE
        WHEN m.temporary_acting IS NOT NULL THEN 'Yes'
        ELSE 'No'
    END AS Temporary_Acting_Head
FROM department d
JOIN management m ON d.Department_ID = m.department_ID
JOIN head h ON m.head_ID = h.head_ID
WHERE d.Budget_in_Billions > 1;
"""
question = "Fetch the name of the departments, the ages of their heads, and whether there is any temporary acting head for departments with a budget exceeding $3 billion."


query = pip_flow.generate_sql(schema=schema, question=question, instructions=instructions, examples=examples)

SELECT d.Name AS Department_Name, h.age AS Head_Age,
    CASE
        WHEN m.temporary_acting IS NOT NULL THEN 'Yes'
        ELSE 'No'
    END AS Temporary_Acting_Head
FROM department d
JOIN management m ON d.Department_ID = m.department_ID
JOIN head h ON m.head_ID = h.head_ID
WHERE d.Budget_in_Billions > 3;

## Generate Docs

This code snippet demonstrates how to utilize the PipFlow class from the pip_flow module to automatically generate a docstring for a given Python code snippet / Function.

#### Code

In [10]:
# code with syntax error
code = """
def add_numbers(a: float, b:float):
    returrn a + b
"""

docs = pip_flow.generate_docs(code=code)


The function `add_numbers` takes two parameters: `a` and `b`. 

- `a` is a float. It represents the first number to be added.
- `b` is a float. It represents the second number to be added.

The function returns a float. The result of the addition of `a` and `b`.

Possible values for `a` and `b` are any numeric values.

The default value of `a` is 0.0 and `b` is 0.0.

The function returns a float.



#### Function

In [11]:
import requests

docs = pip_flow.generate_docs(requests.post)


The function `post` sends a POST request to the specified URL. It takes several parameters, including `url` for the target URL, `data` for sending data in the body of the request, `json` for sending JSON data, and `**kwargs` for passing additional parameters to the `request` function.

The function returns a `Response` object from the `request` function.

**Parameters:**

- `url`: The URL of the server to which the request is sent. It should be a string.
- `data`: The data to be sent in the body of the request. It can be a dictionary, list of tuples, bytes, or file-like object.
- `json`: A JSON serializable Python object to be sent in the body of the request.
- `**kwargs`: Additional parameters to be passed to the `request` function.

**Return Value:**

- A `Response` object from the `request` function.

**Dtypes:**

- `url`: A string.
- `data`: A dictionary, list of tuples, bytes, or file-like object.
- `json`: A dictionary, list of tuples, bytes, or file-like object.
- `**kwargs`: A dictionary.

**Possible Param Values:**

- `url`: A string representing the URL of the server to which the request is sent.
- `data`: A dictionary, list of tuples, bytes, or file-like object representing the data to be sent in the body of the request.
- `json`: A dictionary, list of tuples, bytes, or file-like object representing the JSON data to be sent in the body of the request.
- `**kwargs`: A dictionary containing additional parameters to be passed to the `request` function.

**Default Param Value:**

- `url`: A string representing the URL of the server to which the request is sent.
- `data`: A dictionary, list of tuples, bytes, or file-like object representing the data to be sent in the body of the request.
- `json`: A dictionary, list of tuples, bytes, or file-like

## Generate Function Call


This functionality enables the model to comprehend various types of docstrings / function / code and subsequently prompt a question in natural language regarding the desired function call. The model then produces a Python-parsable function call, which can be executed directly using `exec` or through additional processing steps.

### Function

In [26]:
import requests

question = """
Make a POST request to https://website.co.in, with data '{"name": "John", "age": 30}' and headers 'Content-Type: application/json'.
"""

function_call = pip_flow.generate_function_call(question=question, function=requests.post)

requests.post(url='https://website.co.in', data='{"name": "John", "age": 30}', headers='Content-Type: application/json')

#### Code

In [28]:
code = """
def add_numbers(a: float, b:float):
    return a + b
"""

question = """
I want to add 4.5 and 6.7
"""
function_call = pip_flow.generate_function_call(question=question, code=code)

Generating docstring for the code..




The function `add_numbers` takes two parameters: `a` and `b`. It is designed to add two numbers together. The function takes two parameters, `a` and `b`, which are both of type `float`. The function returns a single value of type `float`.

The function has the following parameters:

- `a`: The first number to be added. It is of type `float`.
- `b`: The second number to be added. It is of type `float`.

The function returns a value of type `float`.

The function has the following parameters with default values:

- `a`: Defaults to `0.0`.
- `b`: Defaults to `0.0`.

The function has the following parameters with possible values:

- `a`: Possible values range from `-1.7976931348623157e+308` to `1.7976931348623157e+308`.
- `b`: Possible values range from `-1.7976931348623157e+308` to `1.7976931348623157e+308`.



add_numbers(a=4.5, b=6.7)

#### Docstring

In [14]:
docstring = """
Function Name: make_get_req
Description: This function is used to make a GET request.
Parameters:
- path (str): The path of the URL to be requested.
- data (dict): The data to be sent in the body of the request.
- flags (dict): The flags to be sent in the request.
- params (dict): The parameters to be sent in the request.
- headers (dict): The headers to be sent in the request.
- not_json_response (bool): OPTIONAL: If set to True, the function will return the raw response content instead of trying to parse it as JSON.
- trailing (str): OPTIONAL: For wrapping slash symbol in the end of string.
- absolute (bool): OPTIONAL: If set to True, the function will not prefix the URL with the base URL.
- advanced_mode (bool): OPTIONAL: If set to True, the function will return the raw response instead of trying to parse it as JSON.
Returns:
- Union[str, dict, list, None]: The response content as a string, a dictionary, a list, or None if the response was not successful.
"""

question = """
Make a GET request for the URL parameter using variable_2. For the params parameter, use 'weight' as one of the keys with variable_3 as its value, and 'width' as another key with a value of 10. For the data parameter, use variable_1. Prefix the URL with the base URL, and ensure the response is in raw format.
"""


function_call = pip_flow.generate_function_call(question=question, docstring=docstring)

make_get_req(path='variable_2', data=variable_1, params={'weight': variable_3}, headers={}, not_json_response=True, absolute=True)

## Generate Plan

### Add functions / callbales to PipFlow class

In [15]:
import requests
from bs4 import BeautifulSoup

functions = [
    requests.get,
    BeautifulSoup,
    BeautifulSoup.find_all,
]

pip_flow.add_callables(
    functions, generate_docs=False
)

pip_flow.functions

[Function(signature='(url, params=None, **kwargs)', docs='Sends a GET request.\n\n    :param url: URL for the new :class:`Request` object.\n    :param params: (optional) Dictionary, list of tuples or bytes to send\n        in the query string for the :class:`Request`.\n    :param \\*\\*kwargs: Optional arguments that ``request`` takes.\n    :return: :class:`Response <Response>` object\n    :rtype: requests.Response\n    ', name='get', full_name='requests.api.get'),
 Function(signature="(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)", docs='A data structure representing a parsed HTML or XML document.\n\n    Most of the methods you\'ll call on a BeautifulSoup object are inherited from\n    PageElement or Tag.\n\n    Internally, this class defines the basic interface called by the\n    tree builders when converting an HTML/XML document into a data\n    structure. The interface abstracts away the differe

### Create a Plan

In [16]:
question = """
Make get request to https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw.
Create Beautiful soup class and find all the divs
"""

plan = pip_flow.generate_plan(question)

{
    "tasks": [
        {
            "task_id": 1,
            "function_name": "get",
            "parameters": [
                {
                    "name": "url",
                    "value": "https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw",
                    "description": "URL of the page to be scraped",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_1"
            ],
            "description": "Fetches the content of the page and converts it into a BeautifulSoup object."
        },
        {
            "task_id": 2,
            "function_name": "BeautifulSoup",
            "parameters": [
                {
                    "name": "markup",
                    "value": "variable_1",
                    "description": "The markup content of the page",
                    "dtype": "string"
                }
            ],
            "outp

### Generate Code from the Plan

In [17]:
pip_flow.plan_to_code()

# Importing required libraries
from bs4 import BeautifulSoup
import requests

# Get the page content
url = 'https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw'
response = requests.get(url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Find all div elements
divs = soup.find_all('div')

# Print the divs
for div in divs:
    print(div)

### Visualise the Plan

In [18]:
pip_flow.visualise_plan()

/usr/local/lib/python3.10/dist-packages/pip_flow/pip_flow.py:399: UserWarning: Visualisation is not supported in VS code yet.
  warnings.warn("Visualisation is not supported in VS code yet.")


network.html


### Create Custom Plan Templates

#### Register template with a unique name

In [19]:
prompt = """
<functions>
{func_info}
</functions>
<json_structure>
{{
  "tasks": [
    {{
      "task_id": 1,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }},
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }}
      ],
      "outputs": ["variable_1"],
      "description": "some description"
    }},
    {{
      "task_id": 2,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }},
        {{,
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }}
      ],
      "outputs": ["variable_2"],
      "description": "some description"
    }}
  ]
}}
</json_structure>
<instructions>
- use self parameter with proper value based on the question.
- name outputs as variable_1 , variable_2 , variable_3 , variable_4 and more variables in chronological order.
- give attention to the type annotation of the parameter given while filling values.
{instructions}
</instructions>
<question>
Given the above functions,
- Do not give the parameters in json which have null values and default values of the function, only give the sequencial function calls with parameters to execute the below question:
{question}
</question>
"""

pip_flow.add_plan_template("template_name", prompt)

for template_name in pip_flow.prompt_templates.keys():
  print(f"Template Name : {template_name}")


Template Name : default
Template Name : template_name


#### Load templates from JSON file

In [20]:
pip_flow.load_templates(filepath="templates.json")

for template_name in pip_flow.prompt_templates.keys():
  print(f"Template Name : {template_name}")

Couldn load template with error [Errno 2] No such file or directory: 'templates.json'
Template Name : default
Template Name : template_name


#### Update Configurations in Templates

- When we create a plan using `generate_plan`, the latest base template and configs are used to substitute the values in the selected template.

- We can override that with the `make_live_prompt` function.

In [35]:
config = {
    "func_info": str(
        [
            f"""--name:{function.name}\n--annotations:{function.signature}\n--doc:{function.docs}\n\n"""
            for function in pip_flow.functions[:2]
        ]
    ),
    "function_list": [function.name for function in pip_flow.functions[:2]],
    "instructions": "Some new Instruction",
}

template_name = "default"

pip_flow.make_live_prompt(template_name, config)

'Updated config and base prompt template successfully.'

In [36]:
print(pip_flow.latest_base_prompt)


<functions>
{func_info}
</functions>
<json_structure>
{{
  "tasks": [
    {{
      "task_id": 1,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }},
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }}
      ],
      "outputs": ["variable_1"],
      "description": "some description"
    }},
    {{
      "task_id": 2,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
       

In [37]:
pip_flow.latest_configs


{'func_info': '[\'--name:get\\n--annotations:(url, params=None, **kwargs)\\n--doc:Sends a GET request.\\n\\n    :param url: URL for the new :class:`Request` object.\\n    :param params: (optional) Dictionary, list of tuples or bytes to send\\n        in the query string for the :class:`Request`.\\n    :param \\\\*\\\\*kwargs: Optional arguments that ``request`` takes.\\n    :return: :class:`Response <Response>` object\\n    :rtype: requests.Response\\n    \\n\\n\', \'--name:BeautifulSoup\\n--annotations:(markup=\\\'\\\', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)\\n--doc:A data structure representing a parsed HTML or XML document.\\n\\n    Most of the methods you\\\'ll call on a BeautifulSoup object are inherited from\\n    PageElement or Tag.\\n\\n    Internally, this class defines the basic interface called by the\\n    tree builders when converting an HTML/XML document into a data\\n    structure. The int

In [41]:
question = """
Make get request to https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw,
Create Beautiful soup class from it
"""

plan = pip_flow.generate_plan(question)

{
    "tasks": [
        {
            "task_id": 1,
            "function_name": "get",
            "parameters": [
                {
                    "name": "url",
                    "value": "https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw",
                    "description": "URL for the new :class:`Request` object.",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_1"
            ],
            "description": "Sends a GET request to the specified URL."
        },
        {
            "task_id": 2,
            "function_name": "BeautifulSoup",
            "parameters": [
                {
                    "name": "markup",
                    "value": "variable_1",
                    "description": "The markup (HTML, XML, etc.) to parse.",
                    "dtype": "string"
                }
            ],
            "outputs": [
        